In [1]:
# The game is a side-scrolling tank game where the player can move, jump, shoot enemies, and collect health. The camera follows the player, and enemies spawn and attack. If the player loses all lives, a game-over screen with a restart button is shown.

import pygame
import random

pygame.init()

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
WORLD_WIDTH = 1600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

background_image = pygame.image.load("/Users/nishchaltamang/Downloads/Background.jpg")
background_image = pygame.transform.scale(background_image, (SCREEN_WIDTH, SCREEN_HEIGHT))

# Camera class

class Camera:
    def __init__(self, width, height):
        self.camera = pygame.Rect(0, 0, width, height)
        self.width = width
        self.height = height

    def apply(self, entity):
        return entity.rect.move(self.camera.topleft)

    def update(self, target):
        x = -target.rect.x + int(SCREEN_WIDTH / 2)
        y = -target.rect.y + int(SCREEN_HEIGHT / 2)
        x = min(0, x)
        x = max(-(self.width - SCREEN_WIDTH), x)
        y = max(-(self.height - SCREEN_HEIGHT), y)
        y = min(0, y)
        self.camera = pygame.Rect(x, y, self.width, self.height)

# Player class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((80, 40), pygame.SRCALPHA)
        self.draw_tank(GREEN)
        self.rect = self.image.get_rect()
        self.rect.x = 100
        self.rect.y = SCREEN_HEIGHT - 100
        self.speed = 5
        self.jump_power = -15
        self.gravity = 1
        self.velocity_y = 0
        self.is_jumping = False
        self.health = 100
        self.lives = 3

    def draw_tank(self, color):
        pygame.draw.rect(self.image, color, (10, 20, 60, 20))
        pygame.draw.rect(self.image, color, (20, 2, 30, 40))
        pygame.draw.rect(self.image, BLACK, (10, 35, 60, 5))
        pygame.draw.rect(self.image, BLACK, (10, 15, 60, 5))

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_UP]:
            self.rect.y -= self.speed
        if keys[pygame.K_LEFT]:
            self.rect.x -= self.speed
        if keys[pygame.K_RIGHT]:
            self.rect.x += self.speed
        if keys[pygame.K_SPACE] and not self.is_jumping:
            self.jump()

        self.rect.y += self.velocity_y
        self.velocity_y += self.gravity
        if self.rect.y >= SCREEN_HEIGHT - 100:
            self.rect.y = SCREEN_HEIGHT - 100
            self.is_jumping = False

    def jump(self):
        self.is_jumping = True
        self.velocity_y = self.jump_power

    def shoot(self):
        bullet = Projectile(self.rect.centerx, self.rect.top)
        all_sprites.add(bullet)
        bullets.add(bullet)

# Projectile class
class Projectile(pygame.sprite.Sprite):
    def __init__(self, x, y, speed=-10):
        super().__init__()
        self.image = pygame.Surface((10, 5))
        self.image.fill(RED)
        self.rect = self.image.get_rect()
        self.rect.centerx = x
        self.rect.centery = y
        self.speed = speed

    def update(self):
        self.rect.y += self.speed
        if self.rect.bottom < 0 or self.rect.top > SCREEN_HEIGHT:
            self.kill()

# Enemy class
class Enemy(pygame.sprite.Sprite):
    def __init__(self, speed_increase=0):
        super().__init__()
        self.image = pygame.Surface((80, 40), pygame.SRCALPHA)
        self.draw_tank(RED)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(100, SCREEN_WIDTH - 100)
        self.rect.y = random.randint(-100, -40)
        self.speed = random.randint(1, 3) + speed_increase
        self.shoot_delay = random.randint(80, 180)

    def draw_tank(self, color):
        pygame.draw.rect(self.image, color, (10, 20, 60, 20))
        pygame.draw.rect(self.image, color, (20, 30, 10, 20))
        pygame.draw.rect(self.image, BLACK, (10, 35, 60, 5))
        pygame.draw.rect(self.image, BLACK, (10, 15, 60, 5))

    def update(self):
        self.rect.y += self.speed
        if self.rect.top > SCREEN_HEIGHT:
            self.kill()
        self.shoot_delay -= 1
        if self.shoot_delay <= 0:
            self.shoot()
            self.shoot_delay = random.randint(30, 120)

    def shoot(self):
        bullet = Projectile(self.rect.centerx, self.rect.bottom, speed=10)
        all_sprites.add(bullet)
        enemy_bullets.add(bullet)

# Collectible class
class Collectible(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((20, 20))
        self.image.fill(BLUE)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, WORLD_WIDTH - 20)
        self.rect.y = random.randint(SCREEN_HEIGHT - 200, SCREEN_HEIGHT - 50)

# Button class for the restart button
class Button:
    def __init__(self, x, y, width, height, text, color, hover_color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.font = pygame.font.SysFont(None, 36)

    def draw(self, screen):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            pygame.draw.rect(screen, self.hover_color, self.rect)
        else:
            pygame.draw.rect(screen, self.color, self.rect)

        text_surface = self.font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)

    def is_clicked(self):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos) and pygame.mouse.get_pressed()[0]:
            return True
        return False

# Main game function to reset the game
def reset_game():
    global player, all_sprites, bullets, enemy_bullets, enemies, collectibles, score, level, enemy_spawn_timer, collectible_spawn_timer, game_over

    player = Player()
    all_sprites = pygame.sprite.Group()
    bullets = pygame.sprite.Group()
    enemy_bullets = pygame.sprite.Group()
    enemies = pygame.sprite.Group()
    collectibles = pygame.sprite.Group()
    all_sprites.add(player)

    score = 0
    level = 1
    enemy_spawn_timer = 100
    collectible_spawn_timer = 200
    game_over = False

# Main game loop
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Tank Side-Scrolling Game")
clock = pygame.time.Clock()

# Initialize game state
reset_game()

font = pygame.font.SysFont(None, 36)

# Create restart button
restart_button = Button(SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2, 200, 50, "Restart", RED, GREEN)

def display_info():
    health_text = font.render(f"Health: {player.health}", True, WHITE)
    lives_text = font.render(f"Lives: {player.lives}", True, WHITE)
    score_text = font.render(f"Score: {score}", True, WHITE)
    level_text = font.render(f"Level: {level}", True, WHITE)
    screen.blit(health_text, (10, 10))
    screen.blit(lives_text, (10, 40))
    screen.blit(score_text, (10, 70))
    screen.blit(level_text, (10, 100))

def check_collisions():
    global score, game_over
    hits = pygame.sprite.groupcollide(enemies, bullets, True, True)
    for hit in hits:
        score += 100

    health_hits = pygame.sprite.spritecollide(player, collectibles, True)
    for hit in health_hits:
        player.health += 20
        if player.health > 100:
            player.health = 100

    enemy_hits = pygame.sprite.spritecollide(player, enemy_bullets, True)
    for hit in enemy_hits:
        player.health -= 20
        if player.health <= 0:
            player.lives -= 1
            player.health = 100
            if player.lives <= 0:
                game_over = True

# Game loop
running = True
while running:
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_s and not game_over:
                player.shoot()

    if not game_over:
        # Game logic
        enemy_spawn_timer -= 1
        if enemy_spawn_timer <= 0:
            enemy = Enemy()
            all_sprites.add(enemy)
            enemies.add(enemy)
            enemy_spawn_timer = random.randint(80, 150)

        collectible_spawn_timer -= 1
        if collectible_spawn_timer <= 0:
            collectible = Collectible()
            all_sprites.add(collectible)
            collectibles.add(collectible)
            collectible_spawn_timer = random.randint(200, 300)

        all_sprites.update()
        camera = Camera(WORLD_WIDTH, SCREEN_HEIGHT)
        camera.update(player)

        check_collisions()

        screen.blit(background_image, (0, 0))

        for sprite in all_sprites:
            screen.blit(sprite.image, camera.apply(sprite))

        display_info()
    else:
        # Game over screen and restart button
        screen.fill(BLACK)
        game_over_text = font.render("Game Over!", True, WHITE)
        screen.blit(game_over_text, (SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 - 100))
        restart_button.draw(screen)
        if restart_button.is_clicked():
            reset_game()

    pygame.display.flip()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Camera Class: Manages the scrolling view in a large game world by tracking the player's position and adjusting the visible game area. The apply method adjusts the positions of entities, and update centers the camera on the player while keeping it within the world boundaries.

Player Class: Represents the player-controlled tank, handling movement, jumping, and shooting. It has attributes for the tank's appearance, position, health, lives, and jumping physics. Functions include drawing the tank, moving/jumping logic, and firing projectiles.

Projectile Class: Manages bullets fired by the player and enemies. It controls the bullets speed, position, and removal when they move off-screen.

Enemy Class: Represents enemy tanks that move downward and shoot bullets. The class manages enemy movement, shooting behavior, and visual representation.


Collectible Class: Represents health pickups for the player. It has attributes for the collectible's appearance and position on the screen.

Button Class: A clickable button for restarting the game when it’s over. It handles the button’s appearance and checks for user clicks.

reset_game(): Resets the game state by creating a new player, resetting score, level, and spawning timers for enemies and collectibles.

display_info(): Displays the player's health, lives, score, and level on the screen.


check_collisions(): Handles collisions between bullets and enemies, the player and collectibles, and the player and enemy bullets, adjusting score, health, and lives.

Main Game Loop: Runs the core game logic, including handling player input, updating game entities, detecting collisions, and displaying game-over and restart options.
